In [1]:
# numerical packages
from copy import copy, deepcopy
import numpy as np
import scipy as sp
from scipy.optimize import minimize

# for visualization (plot and animation)
import matplotlib.pyplot as plt
import matplotlib.animation
from mpl_toolkits.mplot3d import Axes3D

# for drawing Bloch sphere
from qiskit.visualization import plot_bloch_vector
from qiskit.visualization.bloch import Bloch

# my class for the task1
from PSRWST